# Writing Formatted Output

## Overview

### Questions

* How do I display status information during a simulation?
* How can I log user-defined quantities?
* How can I write formatted output to a text file?

### Objectives

* Demonstrate **user-defined log quantities**.
* Explain the use of **Table** to display status information during a simulation run.
* Show that **Table** can write to a file.

## Boilerplate code

In [ ]:
import hoomd
import datetime

Remove files that this notebook will generate.

In [ ]:
import os
fn = os.path.join(os.getcwd(), 'log.txt')
![ -e "$fn" ] && rm log.txt

## Define the Simulation

This tutorial executes the Lennard-Jones particle simulation from a previous tutorial. 
See [*Introducing Molecular Dyamics*](../01-Introducing-Molecular-Dynamics/00-index.ipynb) for a complete description of this code.

In [3]:
gpu = hoomd.device.GPU()
sim = hoomd.Simulation(device=gpu)
sim.create_state_from_gsd(filename='../01-Introducing-Molecular-Dynamics/random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

## Formatted output

The **Table** writer formats log quantities in human-readable text and writes them to `stdout` or a file.
**Table** only supports `scalar` and `string` quantities due to the limitations of this format.
This section shows you how to use **Table** to display status information during a simulation run.

## Add quantities to a Logger

The `categories` argument to **Logger** defines the **categories** that it will accept.

In [4]:
logger = hoomd.logging.Logger(categories=['scalar', 'string'])

Log the and simulation `timestep` and `tps` quantities:

In [5]:
logger.add(sim, quantities=['timestep', 'tps'])

You can also log user-defined quantities using functions, callable class instances, or class properties.
For example, this class computes the estimated time remaining:

In [6]:
class Status():
    def __init__(self, sim):
        self.sim = sim
    
    @property
    def seconds_remaining(self):
        try:
            return (self.sim.final_timestep - self.sim.timestep) / self.sim.tps
        except ZeroDivisionError:
            return 0        
    
    @property
    def etr(self):
        return str(datetime.timedelta(seconds=self.seconds_remaining))


Assign the loggable quantity using the tuple `(object, property_name, flag)`,  where `flag` is the string name of the flag for this quantity. (The tuple for callable objects would be `(callable, flag)`).

In [7]:
status = Status(sim)
logger[('Status', 'etr')] = (status, 'etr', 'string')

Represent the namespace of your user-defined quantity with a tuple of strings - `('Status', 'etr'`) above.
You can use any number of arbitrary strings in the tuple to name your quantity.

## Display quantities with Table

**Table** is a **Writer** that formats the quantities in a **Logger** into a human readable table.
Create one that triggers periodically:

In [8]:
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)

Add it to the simulation:

In [9]:
sim.operations.writers.append(table)

Run the simulation and see the output:

In [10]:
sim.run(100000)

Simulation.timestep  Simulation.tps     Status.etr   
       35000           5265.75804     0:00:18.041087 
       40000           6350.73673     0:00:14.171584 
       45000           7812.27620     0:00:10.880312 
       50000           8838.90728     0:00:09.050893 
       55000           9581.52730     0:00:07.827562 
       60000          10154.75001     0:00:06.893326 
       65000          10618.07132     0:00:06.121639 
       70000          10987.62543     0:00:05.460689 
       75000          11291.99901     0:00:04.870705 
       80000          11558.60503     0:00:04.325782 
       85000          11769.42129     0:00:03.823468 
       90000          11956.99873     0:00:03.345321 
       95000          12126.16781     0:00:02.886320 
      100000          12269.06013     0:00:02.445175 
      105000          12393.47665     0:00:02.017190 
      110000          12511.16049     0:00:01.598573 
      115000          12611.78251     0:00:01.189364 
      120000          12707.

Later in this notebook, you are going to create another **Table Writer**.
Remove `table` now to avoid confusing the two:

In [11]:
sim.operations.writers.remove(table)

## Save Table output to a file

**Table** writes to `stdout` by default.
It can write to a file (or any Python file-like object with `write` and `flush` methods) instead.

Open the file to write:

In [12]:
file = open('log.txt', 'x')

Create a **Table** **Writer** that outputs to this file and add it to the **Simulation**:

In [13]:
table_file = hoomd.write.Table(output=file,
                               trigger=hoomd.trigger.Periodic(period=5000),
                               logger=logger)
sim.operations.writers.append(table_file)

Run the simulation:

In [14]:
sim.run(100000)

You can read the file with standard tools:

In [15]:
!tail log.txt

      185000          14469.17930     0:00:03.110059 
      190000          14473.70326     0:00:02.763633 
      195000          14490.79564     0:00:02.415326 
      200000          14492.44457     0:00:02.070044 
      205000          14493.44820     0:00:01.724917 
      210000          14509.96076     0:00:01.378363 
      215000          14511.57807     0:00:01.033657 
      220000          14513.50513     0:00:00.689013 
      225000          14524.01326     0:00:00.344257 
      230000          14524.47624        0:00:00     


In this section, you have displayed **loggable quantities** during a simulation run and saved them to a text file.
This is the last section in the logging tutorial.

[Previous section](03-Storing-Particle-Shape.ipynb) / [Tutorial index](../README.md)